In [72]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import spacy as sp
import en_core_web_sm
import string
import pickle
import scipy as scp

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import TfidfVectorizer

In [83]:
rdata = pd.read_json('../../data/clean/relevantNewsNLTK4.json')
irdata = pd.read_json('../../data/clean/irrelevantNewsNLTK4.json')
rdata['relevance']=1
irdata['relevance']=0
data = rdata.append(irdata, ignore_index=True)
data.head()

,content,headline,source,summary,uid,index,label,relevance
0,lowly milkshake weapon choice britons determin...,Milkshakes become weapon of choice in UK Europ...,Agence France Presse,Former UK Independence Party leader Nigel Fara...,a437ff48-104a-54bb-bff7-c7a736158524,0,1,1
1,anz race set bring biggest white collar job lo...,ANZ's first assault in the looming job armageddon,News Ltd.,ANZ has moved to the front of the race that’ s...,366c92af-8143-5ffa-8702-4f26bd22c8b6,1,1,1
2,jul 10 carnival cruise stateroom attendants ex...,Carnival Cruise Line to collect your used soap...,Tribune Content Agency,Jul. 10-- Carnival Cruise Line stateroom atten...,863096d4-48f0-5a7c-bee6-384a76d575ee,2,1,1
3,chennai rohit 543 saravanan 546 wickets standa...,Standard CC bags fourth title in a row [New In...,SyndiGate Media Inc.,CHENNAI: R Rohit and P Saravanan took five wic...,3e4d6490-4224-595e-be26-4cb249209b8f,3,1,1
4,donald trump nominee lead fish wildlife servic...,Revealed: Trump's Wildlife Service pick has ti...,Guardian,New revelations show she also has ties to the ...,9f3e248d-b040-5058-bdc9-61c4de59f02a,4,1,1


In [110]:
X = data.drop(['relevance'], axis=1) # the features we want to analyze
y = data['relevance'] # the labels, or answers, we want to test against

# X_train and y_train are the entire dataset (for now)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [111]:
# create tfidf vector based on tokenizer defined above
tfidf_vector = TfidfVectorizer()

In [112]:
def tfidf_transformer(tfidf_vector, data):
    '''
    Input: tfidf vector, data
    Output: transformed data
    '''
    data_tfidf = tfidf_vector.fit_transform(data)
    return data_tfidf

In [113]:
# confirm dimensions of extracted feature is correct
x_train_tfidf = tfidf_transformer(tfidf_vector, X_train['content'])
print(x_train_tfidf.shape)
x_test_tfidf = tfidf_transformer(tfidf_vector, X_test['content'])
print(x_test_tfidf.shape)

(16000, 199293)
(4000, 85283)


In [114]:
y_train = y_train.values.reshape(16000, 1)
y_test = y_test.values.reshape(4000, 1)

In [115]:
train_tfidf = scp.sparse.hstack((x_train_tfidf, y_train))
print(train_tfidf.shape)
test_tfidf = scp.sparse.hstack((x_test_tfidf, y_test))
print(test_tfidf.shape)

(16000, 199294)
(4000, 85284)


In [116]:
# Save pickle files to feature folder in data
pickle.dump(train_tfidf, open("../../data/feature/train_tfidf.pickle", "wb"))
pickle.dump(test_tfidf, open("../../data/feature/test_tfidf.pickle", "wb"))